In [99]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense,LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.layers import Dropout
from tensorflow.keras import metrics
import pandas as pd

In [100]:
user_df = pd.read_json("../data/MOOCCube/entities/user.json",lines=True)

user_df = user_df[["id","course_order"]]
print(user_df.shape)


user_df =user_df[user_df['course_order'].apply(lambda x: len(x) >= 5)]
print(user_df.shape)
data = user_df.to_dict(orient='records')[:1000]
print(data)


(199199, 2)
(34917, 2)
[{'id': 'U_7001215', 'course_order': ['C_course-v1:TsinghuaX+00740043_2x_2015_T2+sp', 'C_course-v1:TsinghuaX+30240184+sp', 'C_course-v1:TsinghuaX+00740043X_2015_T2+sp', 'C_course-v1:TsinghuaX+10421094X_2015_2+sp', 'C_course-v1:TsinghuaX+30240184_2X+sp']}, {'id': 'U_7423998', 'course_order': ['C_course-v1:TsinghuaX+30240184_2X+sp', 'C_course-v1:SCUT+145033+sp', 'C_course-v1:TsinghuaX+00740043X_2015_T2+sp', 'C_course-v1:TsinghuaX+30240184+sp', 'C_course-v1:TsinghuaX+30640014X+sp', 'C_course-v1:TsinghuaX+00690092X+sp', 'C_course-v1:TsinghuaX+30240243X+sp']}, {'id': 'U_545306', 'course_order': ['C_course-v1:TsinghuaX+20430064_2X+sp', 'C_course-v1:TsinghuaX+02070251X+2019_T1', 'C_course-v1:TsinghuaX+0350161X_2015_T2+sp', 'C_course-v1:TsinghuaX+60240013X+sp', 'C_course-v1:TsinghuaX+01030132X+sp', 'C_course-v1:TsinghuaX+20250103X+sp', 'C_course-v1:TsinghuaX+20220214X+2019_T1', 'C_course-v1:TsinghuaX+02070251X+sp', 'C_course-v1:TsinghuaX+20220053X_2015_T2+sp', 'C_course-

In [103]:



course_to_id = {course: idx for idx, course in enumerate(set(course for d in data for course in d["course_order"]))}
num_courses = len(course_to_id)

user_course_matrix = np.zeros((len(data), num_courses), dtype=np.float32)
user_course_predictions = np.zeros((len(data), num_courses), dtype=np.float32)
for i, d in enumerate(data):
    courses = d["course_order"]

    index_partition = int(len(courses) * 0.75)

    courses_1 = courses[:index_partition]
    courses_2 = courses[index_partition:]
    for course in courses_1:
        course_id = course_to_id[course]
        user_course_matrix[i, course_id] = 1.0
        
    for course in courses_2:
        course_id = course_to_id[course]
        user_course_predictions[i, course_id] = 1.0
model = Sequential([
    Embedding(input_dim=num_courses, output_dim=32, input_length=num_courses),
    LSTM(64),
    Dense(num_courses, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', metrics.Precision()])

model.fit(user_course_matrix, user_course_predictions, epochs=10, batch_size=1)





Epoch 1/10


TypeError: in user code:

    File "c:\Users\Usuario\Desktop\8-semestre\Tesis\venv\lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Usuario\Desktop\8-semestre\Tesis\venv\lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Usuario\Desktop\8-semestre\Tesis\venv\lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Usuario\Desktop\8-semestre\Tesis\venv\lib\site-packages\keras\src\engine\training.py", line 1085, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "c:\Users\Usuario\Desktop\8-semestre\Tesis\venv\lib\site-packages\keras\src\engine\training.py", line 1179, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    Fi

In [ ]:
# def recommend_courses_for_user(user_id, model, data, course_to_id):

#     user_index = None
#     for i, d in enumerate(data):
#         if d["id"] == user_id:
#             user_index = i
#             break

#     user_course_probabilities = model.predict(user_course_matrix[user_index:user_index+1])[0]

#     recommendations = {}
#     for course, course_id in course_to_id.items():
#         recommendations[course] = user_course_probabilities[course_id]

#     user_courses = data[user_index]["course_order"]
#     for course in user_courses:
#         recommendations.pop(course, None)

#     sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)

#     return sorted_recommendations


In [ ]:
# user_id_to_recommend = 'U_10359065'
# recommendations = recommend_courses_for_user(user_id_to_recommend, model, data, course_to_id)[:10]

# print(f"Recomendaciones para el usuario {user_id_to_recommend}:")
# recomendations = []
# for course, probability in recommendations:
#     print(f"Libro: {course}, Probabilidad: {probability:.4f}")
#     dic = {"id":course,"probability":probability} 
#     recomendations.append(dic)
# print(recomendations)
    

1/1 [==============================] - 0s 111ms/step
Recomendaciones para el usuario U_10359065:
Libro: C_course-v1:TsinghuaX+00740043X_2015_T2+sp, Probabilidad: 0.0488
Libro: C_course-v1:TsinghuaX+30640014X+sp, Probabilidad: 0.0474
Libro: C_course-v1:TsinghuaX+30700313X+sp, Probabilidad: 0.0302
Libro: C_course-v1:MITx+6_00_1x+sp, Probabilidad: 0.0262
Libro: C_course-v1:TsinghuaX+30240184+sp, Probabilidad: 0.0196
Libro: C_course-v1:TsinghuaX+20220332X+sp, Probabilidad: 0.0171
Libro: C_course-v1:TsinghuaX+20250103X+sp, Probabilidad: 0.0153
Libro: C_course-v1:TsinghuaX+10421075X_2015_2+sp, Probabilidad: 0.0152
Libro: C_course-v1:MIL+MIL2016001+sp, Probabilidad: 0.0142
Libro: C_course-v1:SCUT+145033+sp, Probabilidad: 0.0139
[{'id': 'C_course-v1:TsinghuaX+00740043X_2015_T2+sp', 'probability': 0.04883775}, {'id': 'C_course-v1:TsinghuaX+30640014X+sp', 'probability': 0.04738396}, {'id': 'C_course-v1:TsinghuaX+30700313X+sp', 'probability': 0.030240385}, {'id': 'C_course-v1:MITx+6_00_1x+sp', 'p

In [ ]:
import pandas as pd

# Crear un DataFrame de ejemplo
data = {'user_id': ['usuario1', 'usuario2', 'usuario3'],
        'courses': [['curso1', 'curso2', 'curso3'], ['curso4', 'curso5'], ['curso6']]}

df = pd.DataFrame(data)

df.head()

,user_id,courses
0,usuario1,"[curso1, curso2, curso3]"
1,usuario2,"[curso4, curso5]"
2,usuario3,[curso6]


In [ ]:
df = df[df['courses'].apply(lambda x: len(x) >= 3)]
df.head()

,user_id,courses
0,usuario1,"[curso1, curso2, curso3]"


In [ ]:
# user_course_matrix = np.zeros((len(data), num_courses), dtype=np.float32)
# user_course_predictions = np.zeros((len(data), num_courses), dtype=np.float32)
# partition = 0.75
# for i, d in enumerate(data):
#     courses = d["course_order"]

#     index_partition = int(len(courses) * 0.75)

#     courses_1 = courses[:index_partition]
#     courses_2 = courses[index_partition:]
#     for course in courses_1:
#         course_id = course_to_id[course]
#         user_course_matrix[i, course_id] = 1.0
        
#     for course in courses_2:
#         course_id = course_to_id[course]
#         user_course_predictions[i, course_id] = 1.0